### SMS and Email Spam Classifier using Naive Bayes (only numpy)

#### Naive Bayes
Naive Bayes is a very popular and powerful classifier. One of the reasons it is popular is because it is very efficient. The data processing of the classifier is linear to the number of features since it doesn't involve iterations.<br>

Naive Bayes operates under the assumption that all features are independent and that the data's distribution is known. The latter is not always true but there are ways to estimate the distribution (with the Maximum Likelihood Estimation or MLE).<br><br>
______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______<br>

__All the coding related to feature generation is the same I already made in my previous 'SMS and Email Classifier notebook' where I used Logistic Regression and Gradient Descent. The 'new' material regarding Naive Bayes starts in the section: Naive Bayes Implementation__<br>
______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______<br>

The data was obtained from:<br>
__Dua, D. and Graff, C. (2019). UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection). Irvine, CA: University of California, School of Information and Computer Science.__<br>

Abstract: The SMS Spam Collection is a public set of SMS labeled messages that have been collected for mobile phone spam research.<br>

Examples:<br>
__ham__ What you doing?how are you?<br>
__ham__ Ok lar... Joking wif u oni...<br>
__ham__ dun say so early hor... U c already then say...<br>
__ham__ MY NO. IN LUTON 0125698789 RING ME IF UR AROUND! H*<br>
__ham__ Siva is in hostel aha:-.<br>
__ham__ Cos i was out shopping wif darren jus now n i called him 2 ask wat present he wan lor. Then he started guessing who i was wif n he finally guessed darren lor.<br>
__spam__ FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! ubscribe6GBP/ mnth inc 3hrs 16 stop?txtStop<br>
__spam__ Sunshine Quiz! Win a super Sony DVD recorder if you canname the capital of Australia? Text MQUIZ to 82277. B<br>
__spam__ URGENT! Your Mobile No 07808726822 was awarded a L2,000 Bonus Caller Prize on 02/09/03! This is our 2nd attempt to contact YOU! Call 0871-872-9758 BOX95QU<br>

In [1]:
import numpy as np
from collections import Counter
import csv

#### Loading Data
Loading data first into spam/no spam so I can decide later which features (words/characters/punctuation) to include.

In [2]:
#Reading file
y_spam=list()
y_no_spam=list()
sms_spam=list()
sms_no_spam=list()

#will also change ham/spam labels to -1/1 to make the math faster when measuring accuracy
with open('SMSSpamCollection', 'r') as file:
    reader = csv.reader(file,delimiter='\t')
    for row in reader:
        if row[0]=='ham':
            y_no_spam.append(-1)
            sms_no_spam.append(row[1])
        else:
            y_spam.append(1)
            sms_spam.append(row[1])

In [3]:
def standardize_data(text):
    # This function will perform some changes to the string received. It will first separate
    #   some characters so that the combination of total words can be reduced significantly.
    #   Then it will remove the stopwords that are always present in sms/emails that doesn't
    #   give relevant information.
    #   List of stopwords was taken from: https://gist.github.com/sebleier/554280 but I removed
    #   some words ('no', 'nor' and 'not').
    # Input: 
    # text : string with complete sms or email message
    # Output:
    # modified_text: string separating characters/words in 'separators' list and removing stop words 
    separators=['-','$','£',':','+','?','!','/','>','<','*','@','http','.com','www','(',')',"'",'.']
    stopwords=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    text=text.lower()
    for separator in separators:
        text=text.replace(separator, ' ' + separator + ' ')
    text=text.replace("   ", " ")
    text=text.replace("  ", " ")
    modified_text=str()
    for word in text.split():
        if word not in stopwords:
            modified_text=modified_text + word + ' '
    return modified_text.strip()

#### Feature Processing and Selection
The process I will follow is really 'easy'.
 1. Will get the top 5% of the words/characters with more 'hits' in the spam texts/emails.
 2. Will do the same with the no-spam texts.
 3. Will put both in a set (to remove duplicates) named 'features'.

I decided to get only 5% of the top features (total words/characters were around 5k) initially because it had a great balance of enough data and speed.
As you will see, I made different runs with more and less features.

In [4]:
#will generate 2 'big' string. One with all the spam texts concatenated and the other one with all the no-spam ones.
#this texts are the ones that are going to help me determine the features.
spam_text=standardize_data( " ".join(sms_spam) )
no_spam_text=standardize_data( " ".join(sms_no_spam) )

In [5]:
def feature_decision(spam_text,no_spam_text,top):
    # This function receive 2 texts and will select the top features in each.
    # Input:
    # spam_text : string
    # no_spam_text : string
    # top: proportion of top words/characters to include. Example: 0.05 will be the top 5% of the features
    # Output:
    # features: union of the 2 set of words/characters from the spam and no-spam texts

    total = Counter()
    spam = Counter()
    no_spam = Counter()

    for word in spam_text.split():
        spam[word] += 1
        total[word] += 1
    for word in no_spam_text.split():
        no_spam[word] += 1
        total[word] += 1

    spam_most_common=dict(spam.most_common()[0:int(len(spam)*top)])
    no_spam_most_common=dict(no_spam.most_common()[0:int(len(no_spam)*top)])

    top_spam_features=set()
    top_no_spam_features=set()

    for k,v in spam_most_common.items():
        top_spam_features.add(k)

    for k,v in no_spam_most_common.items():
        top_no_spam_features.add(k)

    return (top_spam_features | top_no_spam_features)

In [6]:
#will run the previous function to select some features
features=feature_decision(spam_text,no_spam_text,0.05)
print('Example of items in the set Features:\n')
print(list(features)[0:12])
print(list(features)[-12:])

Example of items in the set Features:

['probably', 'wait', 'pay', 'wanted', 'next', 'told', 'ready', 'need', 'await', 'everything', 'til', 'together']
['person', 'delivery', '6', 'wen', 'cash', 'year', 'call', 'chance', 'alright', 'sms', 'haf', 'wat']


#### Creates the vectors according to Features

In [7]:
def create_vector(text,features):
    # Function receives a string and returns the 'vector' with the features count in the text. It will count each
    #   feature word in the text
    # Input:
    # text: string
    # features: features set with top words
    # Output:
    # array: 1 array with n dimensions. Dimensions are equivalent to the size of features set.
    
    text=standardize_data(text)
    dict_features = { i : 0 for i in features }
    for i in text.split():
        try:
            dict_features[i]+=1
        except:
            continue
    return np.array(list(dict_features.values()))

__The following process takes a little bit of time. In my computer takes around 15 seconds (Mac 2019).__

In [8]:
def create_X_datasets(y_spam, sms_spam, y_no_spam, sms_no_spam, features):
    # Function randomly creates training and testing datasets.
    #   1: it creates the vectors. 2: concatenates the y's values.  3: adds a column with random values
    #   4: uses the random values to divide the train from the test arrays.
    # Input:
    # y_spam: list of 1's
    # sms_spam: list with sms/emails spam messages
    # y_no_spam: list of -1's
    # sms_no_spam: list with sms/emails no spam messages
    # features: features set (words/characters we will be looking for)
    # Output:
    # x_train: columns same size as the size of the features. Rows depend, but approximately 70% of total.
    # y_train: 1 column. Same amount of rows than x_train. Values -1 or 1.
    # x_test: columns same size as the size of the features. Rows depend, but approximately 70% of total.
    # y_test: 1 column. Same amount of rows than x_test. Values -1 or 1.
    
    #1: creates vector matrix
    for i in range (len(y_spam)):
        feature_vector=create_vector(sms_spam[i],features)
        feature_vector=feature_vector.reshape(1,feature_vector.shape[0])
        if i == 0:
            X_spam=feature_vector
        else:
            X_spam=np.concatenate([X_spam, feature_vector],axis=0)

    for i in range (len(y_no_spam)):
        feature_vector=create_vector(sms_no_spam[i],features)
        feature_vector=feature_vector.reshape(1,feature_vector.shape[0])
        if i == 0:
            X_no_spam=feature_vector
        else:
            X_no_spam=np.concatenate([X_no_spam, feature_vector],axis=0)

    X=np.concatenate([X_spam, X_no_spam],axis=0)

    #2: concatenates y's
    y=np.concatenate([y_spam, y_no_spam],axis=0)
    X_and_y=np.concatenate([X,y.reshape(y.shape[0],1)],axis=1)

    #3: random numbers column
    random_column = np.array(np.random.rand(len(y)))  #create random column so I can divide train/test sets
    random_column = random_column.reshape(random_column.shape[0],1)
    X_and_y=np.concatenate([X_and_y,random_column],axis=1)
    train=X_and_y[X_and_y[:,-1]<=.7]
    test=X_and_y[X_and_y[:,-1]>.7]

    print("Proportion of data used for training:",round(train.shape[0]/(train.shape[0]+test.shape[0]),3 ))

    #Train features and y
    train=train[:,0:train.shape[1]-1]          #eliminate the random column, we don't need it anymore
    y_train=train[:,-1]                        #get y from dataset (which is now the lat column)
    x_train=train[:,0:train.shape[1]-1]        #eliminating the y column, this is now our feature set

    test=test[:,0:test.shape[1]-1]
    y_test=test[:,-1]
    x_test=test[:,0:test.shape[1]-1]

    #following 2 lines were added to make sure values are 0 or 1 for the features
    x_train=np.where(x_train != 0, 1, 0)
    x_test=np.where(x_test != 0, 1, 0)

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test=create_X_datasets(y_spam, sms_spam, y_no_spam, sms_no_spam, features)
print('\nx_train size: ',x_train.shape, '\ty_train size: ',y_train.shape)
print('\nx_test size: ',x_test.shape, '\ty_test size: ',y_test.shape)

Proportion of data used for training: 0.704

x_train size:  (3920, 475) 	y_train size:  (3920,)

x_test size:  (1652, 475) 	y_test size:  (1652,)


#### Naive Bayes Implementation
The following functions will do all the Naive Bayes probability calculations. Since one of the assumptions is that all variables have the same weight

In [9]:
def PY(y):
    # Function calculates the probability of each class (binary). Its basically of calculation  
    #   of the proportion of positive and negative values in the y array.
    # Input
    # y: binary labels. Same amount of rows as X but with only one 'column'
    # Output:
    #    positive: probability p(y=1)
    #    negative: probability p(y=-1)

    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    y = np.concatenate([y, [-1,1]])
    n = len(y)
    pos=(((y == 1)).sum())/(n)
    return pos,1-pos

positive, negative = PY(y_train)
print('Probability of spam:', round(100*positive,2),'%','\nProbability of NOT spam:',round(100*negative,2),'%')

Probability of spam: 13.39 % 
Probability of NOT spam: 86.61 %


In [10]:
def PXY(x,y):
    # Calculates the probability P(X|Y)
    # Input:
    #   x: matrix of features
    #   y: array of labels or classification of those X features
    # Output:
    #   prob_positive: probability vector of p(x|y=1)
    #   prob_negative: probability vector of p(x|y=-1)

    #  '+1 smoothing': add positive/negative samples to avoid division by zero
    n, d = x.shape
    x = np.concatenate([x, np.ones((2,d)), np.zeros((2,d))])
    y = np.concatenate([y, [-1,1,-1,1]])
    n, d = x.shape
    z=np.concatenate((x, y.reshape(n,1)),axis=1)

    X_with_Y_pos = z[z[:, d] == 1]
    X_with_Y_pos=np.delete(X_with_Y_pos, d, 1)
    total_pos=X_with_Y_pos.shape[0]
    prob_positive=X_with_Y_pos.sum(axis=0)/total_pos

    X_with_Y_neg = z[z[:, d] == -1]
    X_with_Y_neg=np.delete(X_with_Y_neg, d, 1)
    total_neg=X_with_Y_neg.shape[0]
    prob_negative=X_with_Y_neg.sum(axis=0)/total_neg
    
    return prob_positive,prob_negative

prob_positive, prob_negative = PXY(x_test,y_test)

In [11]:
def loglikelihood(prob_positive, prob_negative, X_test, Y_test):
    # Creates the table with the log of the distribution (likelihood). It uses log in order to
    #   avoid loosing precision since most of the probabilities can be very small.
    #Input:
    #  prob_positive: conditional probability for the positive class
    #  prob_negative: conditional probabilities for the non-positive class
    #  X_test : features
    #  Y_test : labels (-1 or +1)
    #Output:
    #    loglikelihood of each point in X_test (n)
    #
    n, d = X_test.shape
    loglikelihood = np.zeros(n)

    for i in range(n):
        y=Y_test[i]
        x=X_test[i,:]
        total=0
        for j in range(d):
            if y==1 and x[j]>0:
                total+=np.log(prob_positive[j])
            elif y==1 and x[j]==0:
                total+=np.log(1-prob_positive[j])
            elif y!=1 and x[j]>0:
                total+=np.log(prob_negative[j])
            else:
                total+=np.log(1-prob_negative[j])
        loglikelihood[i]=total
    return loglikelihood

In [12]:
def pred(positive, negative, prob_positive, prob_negative, X_test):
    # Creates the prediction using probabilities. If p(y=1) > p(y=-1) then label is 1, else is -1
    # Input:
    #   positive: probability of being positive
    #   negative: probability of being negative
    #   prob_positive: conditional probabilities for the positive class
    #   prob_negative: conditional probabilities for the negative class
    #   X_test : features
    #Output:
    #    prediction of each X row
    
    n, d = X_test.shape
    logs=(loglikelihood(prob_positive, prob_negative, X_test, np.ones(n))+np.log(positive))>=(loglikelihood(prob_positive, prob_negative, X_test, -np.ones(n))+np.log(negative))
    prediction=logs*1
    for i in range(n):
        if prediction[i]==0:
            prediction[i]=-1
        i+=1
    return prediction

In [13]:
y_pred = pred(positive, negative, prob_positive, prob_negative, x_test)

#### Accuracy of the Naive Bayes Model

In [14]:
def calculate_accuracy(y_pred, y_test):
    # Calculates accuracy of predictions
    # Input:
    # y_pred: prediction labels
    # y_test: real labels in test set
    # Output:
    # accuracy: measured from 0 to 1 and multiplied by 100 to get a percentage
    scores = y_pred
    accuracy = np.mean(y_pred==y_test)
    
    actual_no_pred_no=0
    actual_yes_pred_yes=0
    actual_no_pred_yes=0
    actual_yes_pred_no=0
    
    for i in range(scores.shape[0]):
        if y_test[i]==y_pred[i]:
            if y_test[i]==-1:
                actual_no_pred_no+=1
            else:
                actual_yes_pred_yes+=1
        else:
            if y_test[i]==-1:
                actual_no_pred_yes+=1
            else:
                actual_yes_pred_no+=1
    print('\nCONFUSION MATRIX\n')
    print('\t PREDICTED')
    print('ACTUAL\t NO\tYES')
    print('------------------------------')
    print('NO SPAM\t',actual_no_pred_no,'\t',actual_no_pred_yes)
    print('SPAM\t',actual_yes_pred_no,'\t',actual_yes_pred_yes)
    print('\n')
    print("Error to minimize (Not spam classified as spam):",round((100*actual_no_pred_yes)/(actual_no_pred_no+actual_yes_pred_yes+actual_no_pred_yes+actual_yes_pred_no),2),'%')

    return accuracy

In [15]:
print('Features: ', len(features), '\t\tAccuracy (%): ',round(100*calculate_accuracy(y_pred,y_test),2))


CONFUSION MATRIX

	 PREDICTED
ACTUAL	 NO	YES
------------------------------
NO SPAM	 1426 	 3
SPAM	 16 	 207


Error to minimize (Not spam classified as spam): 0.18 %
Features:  475 		Accuracy (%):  98.85


__Accuracy of 98.85% is good, specially the low (0.18%) of no spam messages classified as spam.__

#### Additional Analysis with Different Amount of Features

In [16]:
#Run with around TWICE as many features
features=feature_decision(spam_text,no_spam_text,0.1)
x_train, y_train, x_test, y_test=create_X_datasets(y_spam, sms_spam, y_no_spam, sms_no_spam, features)
prob_positive, prob_negative = PXY(x_test,y_test)
y_pred=pred(positive, negative, prob_positive, prob_negative, x_test)
print('Features: ', len(features), '\t\tAccuracy (%): ',round(100*calculate_accuracy(y_pred,y_test),2))

Proportion of data used for training: 0.703

CONFUSION MATRIX

	 PREDICTED
ACTUAL	 NO	YES
------------------------------
NO SPAM	 1426 	 1
SPAM	 23 	 203


Error to minimize (Not spam classified as spam): 0.06 %
Features:  928 		Accuracy (%):  98.55


In [17]:
#Run with around HALF the features as in the first run
features=feature_decision(spam_text,no_spam_text,0.005)
x_train, y_train, x_test, y_test=create_X_datasets(y_spam, sms_spam, y_no_spam, sms_no_spam, features)
prob_positive, prob_negative = PXY(x_test,y_test)
y_pred=pred(positive, negative, prob_positive, prob_negative, x_test)
print('Features: ', len(features), '\t\tAccuracy (%): ',round(100*calculate_accuracy(y_pred,y_test),2))

Proportion of data used for training: 0.702

CONFUSION MATRIX

	 PREDICTED
ACTUAL	 NO	YES
------------------------------
NO SPAM	 1424 	 21
SPAM	 46 	 169


Error to minimize (Not spam classified as spam): 1.27 %
Features:  41 		Accuracy (%):  95.96


In [18]:
#Run with a MINIMAL amount of features
features=feature_decision(spam_text,no_spam_text,0.001)
x_train, y_train, x_test, y_test=create_X_datasets(y_spam, sms_spam, y_no_spam, sms_no_spam, features)
prob_positive, prob_negative = PXY(x_test,y_test)
y_pred=pred(positive, negative, prob_positive, prob_negative, x_test)
print('Features: ', len(features), '\t\tAccuracy (%): ',round(100*calculate_accuracy(y_pred,y_test),2))

Proportion of data used for training: 0.705

CONFUSION MATRIX

	 PREDICTED
ACTUAL	 NO	YES
------------------------------
NO SPAM	 1380 	 43
SPAM	 152 	 67


Error to minimize (Not spam classified as spam): 2.62 %
Features:  7 		Accuracy (%):  88.12


#### Conclusion
The Naive Bayes classifier is considerably faster than the Logistic Regression with Gradient Descent classifier used for this same example.
The main reason is that it calculates the probabilities once and then uses the table to estimate if it is positive or negative.

We get a really good prediction accuracy and with more features we can get even better results. During the implementation of Naive Bayes we can incorporate considerably more features than other classifiers that iterate.